In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas

In [2]:
bike_orderlines_df = pd.read_excel("../00_data/bike_sales/data_wrangled/bike_orderlines.xlsx")

In [5]:
bike_orderlines_df.hvplot.table(width=1200)

:Table   [order_date,order_id,order_line,quantity,price,total_price,model,category_1,category_2,frame_material,bikeshop_name,city,state]

# Get Customer Trends

In [18]:
customer_trends_df = (
    bike_orderlines_df
    .filter(['bikeshop_name', 'model', 'quantity'])
    .groupby(['bikeshop_name', 'model'])
    .agg(quantity_purchase = ('quantity', 'sum'))
    .reset_index()
    .assign(total=lambda x: x.groupby('bikeshop_name')['quantity_purchase'].transform('sum'))
    .assign(prop_of_total=lambda x: x.quantity_purchase / x.total)
    .drop(['quantity_purchase', 'total'], axis='columns')
)

In [19]:
customer_trends_df.hvplot.table()

:Table   [bikeshop_name,model,prop_of_total]

# Convert to Customer-Product Format

In [61]:
customer_product_df = (
    customer_trends_df
    .pivot(index='bikeshop_name', columns='model', values='prop_of_total')
    .reset_index()
    .fillna(0)
)

In [64]:
customer_product_df.head()

model,bikeshop_name,Bad Habit 1,Bad Habit 2,Beast of the East 1,Beast of the East 2,Beast of the East 3,CAAD Disc Ultegra,CAAD12 105,CAAD12 Black Inc,CAAD12 Disc 105,...,Synapse Sora,Trail 1,Trail 2,Trail 3,Trail 4,Trail 5,Trigger Carbon 1,Trigger Carbon 2,Trigger Carbon 3,Trigger Carbon 4
0,Albuquerque Cycles,0.017483,0.006993,0.01049,0.010490,0.003497,0.013986,0.006993,0.000000,0.013986,...,0.000000,0.000000,0.003497,0.006993,0.017483,0.010490,0.006993,0.003497,0.006993,0.006993
1,Ann Arbor Speed,0.006645,0.009967,0.01495,0.009967,0.003322,0.026578,0.014950,0.016611,0.014950,...,0.009967,0.009967,0.014950,0.009967,0.003322,0.011628,0.000000,0.000000,0.000000,0.011628
2,Austin Cruisers,0.008130,0.004065,0.00813,0.008130,0.000000,0.020325,0.020325,0.004065,0.024390,...,0.020325,0.016260,0.016260,0.016260,0.008130,0.008130,0.000000,0.000000,0.000000,0.016260
3,Cincinnati Speed,0.005115,0.000000,0.00000,0.000000,0.000000,0.015345,0.010230,0.015345,0.007673,...,0.012788,0.000000,0.002558,0.002558,0.002558,0.000000,0.010230,0.007673,0.010230,0.020460
4,Columbus Race Equipment,0.010152,0.000000,0.00000,0.005076,0.002538,0.010152,0.027919,0.027919,0.025381,...,0.015228,0.002538,0.002538,0.005076,0.000000,0.000000,0.010152,0.005076,0.017766,0.005076


# Modeling: K-Means

In [107]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=100).fit(customer_product_df.drop('bikeshop_name', axis=1))
kmeans.inertia_

0.09479025820547679

`Inertia` here is the same as the `tot.withinss` in the `kmeans` package in R.

In order to use the elbow method to pick up the optimum number of clusters (k)
to segment the base, we are going to run kmeans for several k's

In [81]:
k_inertias = pd.DataFrame(columns=['k', 'inertia'])

for k in range(1, 16):
    kmeans = KMeans(n_clusters=k, n_init=100).fit(customer_product_df.drop('bikeshop_name', axis=1))
    k_inertias = k_inertias.append({'k': k, 'inertia':kmeans.inertia_}, 
                                   ignore_index=True)

In [85]:
k_inertias.hvplot.table()

:Table   [k,inertia]

In [104]:
import altair as alt

alt.Chart(k_inertias).mark_line(point=True).encode(
    x='k',
    y='inertia', 
    tooltip=['inertia', 'k']
).configure_point(
    size=100
).properties(
    width=500,
    height=200
).interactive()

alt.Chart(...)

Based on the Skree Plot above, we select 4 clusters to segment the customer base.